# 📓 The GenAI Revolution Cookbook

**Title:** How to Build a Knowledge Graph Chatbot with Neo4j, Chainlit, GPT-4o

**Description:** Ship a Python knowledge graph chatbot using Neo4j, Chainlit, and GPT-4o—auto-generate Cypher, visualize results, and answer complex data questions accurately.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Knowledge graph chatbots combine the precision of structured data with the flexibility of natural language. This guide walks you through building a production-ready Python chatbot that queries a Neo4j graph database using GPT-4o, CrewAI for agent orchestration, and Chainlit for the UI. By the end, you'll have a working app that answers questions, generates Cypher queries, and renders interactive charts—all runnable locally with minimal setup.

## What You're Building

A conversational interface that:

- Accepts natural language questions about your graph
- Generates and executes Cypher queries via GPT-4o
- Returns structured results as text, tables, and Plotly charts
- Maintains conversation context across multiple turns

You'll integrate Neo4j for graph storage, CrewAI for agent orchestration (providing built-in memory and standardized tool calling), and Chainlit for a chat UI. This architecture is extensible: add more agents, tools, or data sources as your use case grows.

## Prerequisites

- Python 3.10+
- Neo4j Aura account (free tier) or local Neo4j instance
- OpenAI API key with GPT-4o access
- Basic familiarity with Python, environment variables, and command-line tools

## Setup & Installation

Create a project directory and virtual environment:

In [ ]:
mkdir kg-chatbot && cd kg-chatbot
python -m venv venv
source venv/bin/activate  # On Windows: venv\Scripts\activate

Install dependencies with pinned versions to ensure compatibility:

In [ ]:
pip install chainlit==1.0.200 neomodel==5.2.1 plotly==5.18.0 crewai==0.28.0 pyyaml==6.0.1 python-dotenv==1.0.0 openai==1.12.0 neo4j==5.16.0

Create a `.env` file in the project root with your credentials:

In [ ]:
OPENAI_API_KEY=sk-...
NEO4J_URI=neo4j+s://your-instance.databases.neo4j.io
NEO4J_USER=neo4j
NEO4J_PASSWORD=your-password
NEO4J_DATABASE=neo4j

## Project Structure

Organize your code into the following files:

In [ ]:
kg-chatbot/
├── .env
├── tools/
│   ├── __init__.py
│   ├── db.py
│   ├── query_knowledge_graph.py
│   └── _demo.py
├── agents.yaml
├── tasks.yaml
├── agents.py
└── chat.py

Create the `tools/` directory and an empty `__init__.py`:

In [ ]:
mkdir tools
touch tools/__init__.py

## Step 1: Initialize the Neo4j Connection

This module loads environment variables, converts the Neo4j URI to a Bolt-compatible format for neomodel, and tests connectivity. Using neomodel simplifies Cypher execution and provides a consistent interface for future ODM features if needed.

Create `tools/db.py`:

In [ ]:
import os
import re
from dotenv import load_dotenv
from neomodel import config, db

load_dotenv()

def _to_bolt_uri(uri: str) -> str:
    return re.sub(r"^neo4j\+s", "bolt+s", uri)

def init_db():
    uri = os.getenv("NEO4J_URI")
    user = os.getenv("NEO4J_USER")
    password = os.getenv("NEO4J_PASSWORD")
    database = os.getenv("NEO4J_DATABASE", "neo4j")

    if not uri or not user or not password:
        raise RuntimeError("Missing Neo4j env vars: NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD")

    bolt_uri = _to_bolt_uri(uri)
    config.DATABASE_URL = f"{bolt_uri}/{database}"
    config.AUTH = (user, password)

def test_connection() -> bool:
    try:
        res, meta = db.cypher_query("RETURN 'Connection successful' AS msg")
        return res[0][0] == "Connection successful"
    except Exception as e:
        print(f"Neo4j connection failed: {e}")
        return False

if __name__ == "__main__":
    init_db()
    print("OK" if test_connection() else "FAIL")

Test the connection:

In [ ]:
python tools/db.py

You should see `OK` if credentials are correct.

## Step 2: Seed Sample Data

To validate the chatbot end-to-end, seed a small synthetic dataset. This creates Person, Company, and Product nodes with relationships.

Run the following Cypher in the Neo4j Browser or via a Python script:

```cypher
CREATE (alice:Person {name: 'Alice', age: 30})
CREATE (bob:Person {name: 'Bob', age: 35})
CREATE (acme:Company {name: 'Acme Corp', industry: 'Tech'})
CREATE (widget:Product {name: 'Widget', category: 'Hardware'})
CREATE (alice)-[:WORKS_AT]->(acme)
CREATE (bob)-[:WORKS_AT]->(acme)
CREATE (alice)-[:PURCHASED]->(widget)
CREATE (acme)-[:MAKES]->(widget)
```

Verify with:

```cypher
MATCH (n) RETURN count(n) AS node_count
```

You should see at least 4 nodes.

## Step 3: Build the Query Tool

This tool encapsulates LLM-powered Cypher generation and execution. It accepts a natural language question, generates Cypher with GPT-4o, executes it via neomodel, and returns structured data. This keeps query logic reusable for any agent. If you're interested in building robust pipelines for extracting structured data from language models, our guide on [structured data extraction with LLMs](/article/structured-data-extraction-with-llms-how-to-build-a-pipeline) offers a deep dive into best practices for production-ready systems.

Create `tools/query_knowledge_graph.py`:

In [ ]:
import os
import json
import textwrap
import re
from typing import Any, Dict, List, Optional, Tuple

from openai import OpenAI
from neomodel import db

DEFAULT_SCHEMA = """
Node labels:
- Person(name, age)
- Company(name, industry)
- Product(name, category)
Relationships:
- (Person)-[:WORKS_AT]->(Company)
- (Person)-[:PURCHASED]->(Product)
- (Company)-[:MAKES]->(Product)
"""

class KnowledgeGraphQueryTool:
    def __init__(self, schema: Optional[str] = None, model: str = "gpt-4o"):
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise RuntimeError("OPENAI_API_KEY not set")
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.schema = schema or DEFAULT_SCHEMA

    def generate_cypher(self, question: str) -> str:
        system = "You are a Cypher expert. Generate a single Cypher query. Do not include explanations."
        prompt = f"""
        Given this Neo4j schema:
        {self.schema}

        User question:
        {question}

        Constraints:
        - Return concise columns with clear names.
        - Prefer COUNT(), LIMIT, and WHERE for performance.
        - Only output Cypher, no prose.
        """
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": textwrap.dedent(prompt).strip()}
            ],
            temperature=0.1,
        )
        raw = resp.choices[0].message.content.strip()
        cypher = re.sub(r"^

cypher\s*", "", raw, flags=re.IGNORECASE)
        cypher = re.sub(r"\s*

In [ ]:
$", "", cypher)
        return cypher.strip()

    def execute_cypher(self, cypher: str) -> Tuple[List[Dict[str, Any]], List[str]]:
        results, meta = db.cypher_query(cypher)
        headers = [m['name'] for m in meta]
        rows = [dict(zip(headers, row)) for row in results]
        return rows, headers

    def run(self, question: str) -> Dict[str, Any]:
        try:
            cypher = self.generate_cypher(question)
            rows, headers = self.execute_cypher(cypher)
            return {
                "ok": True,
                "cypher": cypher,
                "headers": headers,
                "rows": rows
            }
        except Exception as e:
            print(f"Error in KnowledgeGraphQueryTool.run: {e}")
            return {
                "ok": False,
                "error": str(e)
            }

Test the tool with a demo script. Create `tools/_demo.py`:

In [ ]:
from tools.db import init_db
from tools.query_knowledge_graph import KnowledgeGraphQueryTool

if __name__ == "__main__":
    init_db()
    tool = KnowledgeGraphQueryTool()
    print(tool.run("How many Person nodes are there?"))

Run:

In [ ]:
python tools/_demo.py

You should see a result with `ok: True` and a count of 2.

## Step 4: Define the Agent and Task

CrewAI provides built-in memory, standardized tool calling, and a clear separation between agent configuration and task logic. This makes it easier to extend with multiple agents or tools later. We'll define the agent and task in YAML for clarity and maintainability.

Create `agents.yaml`:

```yaml
graph_analyst:
  role: "Graph Data Analyst"
  goal: "Answer user questions by querying the Neo4j knowledge graph accurately and concisely."
  backstory: "You are an expert in graph databases and Cypher. You translate natural language into precise queries and present results clearly."
  tone: "Professional, concise, and helpful."
  instructions: |
    - Use the query_knowledge_graph tool to answer questions.
    - If the tool returns an error, explain it to the user and suggest rephrasing.
    - If results are empty, state that clearly.
    - When appropriate, suggest a chart to visualize the data.
  schema: |
    Node labels:
    - Person(name, age)
    - Company(name, industry)
    - Product(name, category)
    Relationships:
    - (Person)-[:WORKS_AT]->(Company)
    - (Person)-[:PURCHASED]->(Product)
    - (Company)-[:MAKES]->(Product)
```

Create `tasks.yaml`:

```yaml
answer_question:
  description: |
    Answer the user's question using the knowledge graph.
    User question: {question}
    Conversation history:
    {history}
  expected_output: |
    A clear, concise answer with supporting data.
    If a chart is appropriate, include a JSON block with chart spec:
```

json
    {
      "chart": {
        "type": "bar",
        "title": "Chart Title",
        "x": ["label1", "label2"],
        "y": [10, 20]
      }
    }

Create `agents.py` to load these configs and build CrewAI components:

In [ ]:
import yaml
from pathlib import Path
from typing import Any, Dict

from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool

def load_yaml(path: str) -> Dict[str, Any]:
    return yaml.safe_load(Path(path).read_text())

def build_graph_analyst() -> Agent:
    data = load_yaml("agents.yaml")["graph_analyst"]
    return Agent(
        role=data["role"],
        goal=data["goal"],
        backstory=data["backstory"],
        allow_delegation=False,
        verbose=True,
        memory=True,
        tools=[],
    )

def build_answer_task() -> Task:
    data = load_yaml("tasks.yaml")["answer_question"]
    return Task(
        description=data["description"],
        expected_output=data["expected_output"],
        agent=None,
    )

def build_crew(agent: Agent, task: Task, tools: list) -> Crew:
    agent.tools = tools
    task.agent = agent
    return Crew(
        agents=[agent],
        tasks=[task],
        process=Process.sequential,
        verbose=True
    )

## Step 5: Wrap the Tool for CrewAI

CrewAI requires tools to inherit from `BaseTool`. This wrapper exposes the `KnowledgeGraphQueryTool` to the agent.

Add this class to `agents.py`:

In [ ]:
import json
from tools.query_knowledge_graph import KnowledgeGraphQueryTool

class ToolWrapper(BaseTool):
    name: str = "query_knowledge_graph"
    description: str = "Generate and execute Cypher to answer questions about the Neo4j graph."

    def __init__(self, tool: KnowledgeGraphQueryTool):
        super().__init__()
        self.tool = tool

    def _run(self, input: str) -> str:
        result = self.tool.run(input)
        return json.dumps(result)

## Step 6: Build the Chainlit Chat Interface

This module integrates CrewAI, Neo4j, and Plotly into a conversational UI. It maintains conversation history in the user session, passes it to the agent for context-aware responses, and renders charts and tables.

Create `chat.py`:

In [ ]:
import os

required_keys = ["OPENAI_API_KEY", "NEO4J_URI", "NEO4J_USER", "NEO4J_PASSWORD"]
missing = [k for k in required_keys if not os.getenv(k)]
if missing:
    raise EnvironmentError(
        f"Missing required environment variables: {', '.join(missing)}\n"
        "Please set them before running the app."
    )

import re
import json
from typing import Any, Dict, Optional

import chainlit as cl
import plotly.express as px

from tools.db import init_db
from tools.query_knowledge_graph import KnowledgeGraphQueryTool
from agents import build_graph_analyst, build_answer_task, build_crew, ToolWrapper

def extract_chart_json(text: str) -> Optional[Dict[str, Any]]:
    code_blocks = re.findall(r"

json(.*?)

In [ ]:
", text, re.DOTALL | re.IGNORECASE)
    for block in code_blocks:
        try:
            data = json.loads(block.strip())
            if isinstance(data, dict) and "chart" in data:
                chart = data["chart"]
                if "type" in chart and ("x" in chart or "labels" in chart):
                    return chart
        except Exception:
            continue
    return None

def extract_cypher(text: str) -> Optional[str]:
    blocks = re.findall(r"

cypher(.*?)

In [ ]:
", text, re.DOTALL | re.IGNORECASE)
    return blocks[0].strip() if blocks else None

def render_chart(chart_spec: Dict[str, Any]):
    ctype = chart_spec.get("type", "bar")
    title = chart_spec.get("title", "")
    if ctype == "bar":
        fig = px.bar(x=chart_spec.get("x", []), y=chart_spec.get("y", []), title=title)
    elif ctype == "line":
        fig = px.line(x=chart_spec.get("x", []), y=chart_spec.get("y", []), title=title)
    elif ctype == "pie":
        fig = px.pie(names=chart_spec.get("labels", []), values=chart_spec.get("values", []), title=title)
    else:
        fig = px.bar(x=[], y=[], title=f"Unsupported chart type: {ctype}")
    return fig

@cl.on_chat_start
async def on_start():
    init_db()
    tool = KnowledgeGraphQueryTool()
    agent = build_graph_analyst()
    task = build_answer_task()
    crew = build_crew(agent, task, tools=[ToolWrapper(tool)])
    cl.user_session.set("crew", crew)
    cl.user_session.set("history", [])
    await cl.Message(content="Knowledge Graph Chatbot ready. Ask a question about your graph.").send()

@cl.on_message
async def on_message(message: cl.Message):
    crew = cl.user_session.get("crew")
    history = cl.user_session.get("history", [])
    user_input = message.content

    history.append(f"User: {user_input}")
    history_blob = "\n".join(history[-10:])

    try:
        result = crew.kickoff(
            inputs={
                "question": user_input,
                "history": history_blob
            }
        )
        text = result.raw if hasattr(result, "raw") else str(result)
    except Exception as e:
        await cl.Message(content=f"Agent error: {e}").send()
        return

    history.append(f"Assistant: {text}")
    cl.user_session.set("history", history)

    elements = []

    chart = extract_chart_json(text)
    if chart:
        fig = render_chart(chart)
        elements.append(cl.Plotly(name="chart", figure=fig))

    cypher = extract_cypher(text)
    if cypher:
        text += "\n\nExecuted Cypher:\n

cypher\n" + cypher + "\n

In [ ]:
"

    await cl.Message(content=text, elements=elements).send()

## Run and Validate

Start the Chainlit app:

In [ ]:
chainlit run chat.py -w

Open the URL shown in your terminal (usually `http://localhost:8000`). Try these test queries:

- "How many Person nodes are there?"
- "Who works at Acme Corp?"
- "Show me a chart of people by age."

You should see:

- Natural language answers
- Generated Cypher queries (appended to responses)
- Interactive Plotly charts when appropriate

If the tool returns an error, the agent will surface it with a suggestion to rephrase.

## Conversation Memory Strategy

Use short sliding windows and summaries to keep token counts low and context relevant. Store state server-side and avoid sending entire histories unnecessarily. For more on how LLMs handle and sometimes lose context as conversations grow, see our article on [context rot and LLM memory limitations](/article/context-rot-why-llms-forget-as-their-memory-grows).

The current implementation maintains the last 10 messages in `cl.user_session`. For longer conversations, consider:

- Summarizing older messages with GPT-4o
- Storing only the last N turns plus a summary
- Using a vector store to retrieve relevant past exchanges

## Query Caching

Cache frequent Cypher results keyed by normalized queries and parameters. For time-varying data, set short TTLs. This reduces cost and latency for repeated questions. To learn how semantic caching can further optimize your LLM-powered apps, check out our tutorial on [semantic caching with Redis Vector](/article/semantic-cache-llm-how-to-implement-with-redis-vector-to-cut-costs).

Add a simple in-memory cache to `KnowledgeGraphQueryTool`:

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=128)
def _cached_execute(self, cypher: str):
    return self.execute_cypher(cypher)

For production, use Redis or Memcached with TTLs.

## Security Considerations

Executing generated Cypher directly poses risks. Add safeguards:

- Allowlist node labels and relationship types
- Use read-only database credentials
- Parameterize user-supplied literals
- Validate generated queries against a schema

Example allowlist check:

In [ ]:
ALLOWED_LABELS = {"Person", "Company", "Product"}
ALLOWED_RELS = {"WORKS_AT", "PURCHASED", "MAKES"}

def validate_cypher(cypher: str) -> bool:
    labels = re.findall(r":(\w+)", cypher)
    rels = re.findall(r"\[:(\w+)\]", cypher)
    return all(l in ALLOWED_LABELS for l in labels) and all(r in ALLOWED_RELS for r in rels)

Call this before executing queries and reject invalid ones.

## Next Steps

- Add multi-agent workflows for complex queries
- Integrate vector search for hybrid retrieval
- Deploy to production with authentication and rate limiting
- Extend the schema and seed larger datasets

You now have a working knowledge graph chatbot that combines LLM reasoning with structured data precision. Use this foundation to build domain-specific assistants for customer support, analytics, or internal tooling.